Column that I want to use: <br />
Twitter Data<br />
created_at | id | User.screen_name | Text | Lang | retweet_count | reply_count | quote_count | retweeted_status | <br /><br />

Sentiment Analysis<br />
sentiment polarity |

BotOMeter<br />
screen_name | english | 


In [1]:
import tarfile
import json
import gzip
import pandas as pd
import re
from textblob import TextBlob
from pandas.io.json import json_normalize

def clean_tweet(tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())


<h1>Data Preparation</h1>
<h3>Data Preparation #1: Import twitter data</h3>

In [230]:
tar = tarfile.open("../input/2017-09-22.tar.gz", "r:gz")
data = pd.DataFrame()
count=0

'''
        Extracting from twitter data.
'''

for members in tar.getmembers():
    if (count==13):
        f = tar.extractfile(members)
        data = data.append(pd.read_json(f, lines=True))
    count = count + 1


In [314]:
'''
        Compiling hashtags result.
'''
count=0
hashtags=[]
for ent in data['entities']:
    str_hsh=''
    for hsh in ent['hashtags']:
        #print(hsh['text'])
        str_hsh=hsh['text']+';'+str_hsh
    hashtags.append(str_hsh.upper())
    count = count+1
    
'''
        Compiling user screen_name result.
'''
namelist=[]
for x in data['user']:
    namelist.append(x['screen_name'])

twt_data=pd.DataFrame({'screen_name':namelist, 'lang':data['lang'] 
                            ,'retweet_count':data['retweet_count']
                            ,'reply_count':data['reply_count']
                            ,'quote_count':data['quote_count']
                            ,'retweeted_status':data['retweeted_status']
                            ,'hashtags':hashtags
                            ,'created_at':data['created_at']
                           })

In [383]:
data.iloc[0]
#len(twt_data)

contributors                                                               NaN
coordinates                                                               None
created_at                                                 2017-09-22 12:05:02
display_text_range                                                    [0, 140]
entities                     {u'user_mentions': [], u'symbols': [], u'hasht...
extended_entities                                                          NaN
extended_tweet               {u'display_text_range': [0, 135], u'entities':...
favorite_count                                                               0
favorited                                                                False
filter_level                                                               low
geo                                                                       None
id                                                          911199659657498625
id_str                                              

<h3>Data Preparation #2: Import data from previous BotOMeter preprocessing </h3>

In [232]:
bot_data = pd.DataFrame()

for i in range(1,8):
    output="outputbot_"+str(i)+".csv"
    bot_data=bot_data.append(pd.read_csv(output))[['screen_name','english']]

In [233]:
print(len(bot_data))

3232


<h3>Data Preparation #3: Import Sentiment based on Naive Bayes data</h3>
since the analysis process taking time, I decided to separate this process to another notebook file. 

In [234]:
sentiment_data = pd.DataFrame()
sentiment_data = pd.read_csv("sentiment_output.csv")

In [362]:
len(sentiment_data)
data.iloc[185]['text']
#data.iloc[sentiment_data[sentiment_data['p_pos']>0.99].index]['text']


u"As we wait anxiously for Mrs May's #Florence speech, has any Prime Minister in modern history inherited such a Herculean task as #Brexit?"

<h1>Data Merging: Merging the data sources</h1>
since the analysis process taking time, I decided to separate this process to another notebook file. 

In [401]:
sen_dat=twt_data.join(sentiment_data[['class','p_neg','p_pos']])

compiled_data=pd.merge(sen_dat,bot_data, how='left', on=['screen_name'])
print(len(compiled_data))

#compiled_data

4369


<h3>Data checking process: </h3>
since the analysis process taking time, I decided to separate this process to another notebook file. 

In [332]:
assert (len(compiled_data)==len(data)), "Input data and output data has a different count"

<h1>Describing the Data</h1>
since the analysis process taking time, I decided to separate this process to another notebook file. 


<h1>Analysis Part</h1>
since the analysis Part taking time, I decided to separate this process to another notebook file. 

<h3>Analysis 1: sentiment analysis of hashtags</h3>

In [437]:
txt_tag=[]
sntment_tag=[]
for x in range(len(compiled_data)):
    oo=TextBlob(compiled_data.iloc[x]['hashtags']).words
    for p in oo:
        txt_tag.append(p)
        sntment_tag.append(compiled_data.iloc[x]['p_pos'])
    

In [529]:
hashtag_analysis=pd.DataFrame({
            'hashtag':txt_tag,
            'sentimenttag':sntment_tag
            })


lol=hashtag_analysis.groupby(['hashtag'], as_index=False).mean()
lol1=hashtag_analysis.groupby(['hashtag'], sort=True, as_index=False).count().sort_values(by='sentimenttag', ascending=False)
#lol.index.names = ['hashtag']


analysis1=pd.merge(lol,lol1, how='left', on=['hashtag'])
analysis1=analysis1.rename(columns = {
                            'sentimenttag_y':'count',
                            'sentimenttag_x':'p_pos'
                           })


In [532]:
analysis1.sort_values(by=['count'], ascending=False)[0:50]

,hashtag,p_pos,count
199,BREXIT,0.704155,1858
556,EU,0.676697,745
1385,REFERENDUM,0.655990,434
575,EUROPE,0.795995,294
654,FLORENCESPEECH,0.778298,230
653,FLORENCE,0.917295,198
1572,STOPBREXIT,0.696550,181
1820,WALKAWAYMAY,0.702781,163
1399,REMAIN,0.678516,116
1638,THERESAMAY,0.672287,109


In [363]:
#len(compiled_data[pd.isnull(compiled_data['retweeted_status'])==True])
#compiled_data[pd.isnull(compiled_data['english'])==True]

In [ ]:
mydict=Counter(namelist)

for key, value in sorted(mydict.iteritems(), key=lambda (k,v): (v,k), reverse=True):
    print "%s: %s" % (key, value)

<h1>Hypotheses</h1>

<h3>hypothesis #1:</h3>
Does the sentiment analysis in brexit like negative or positive exactly explain the context of the brexit?what is the meaning of positive sentiment in this case??
naives bayes method classification

<h3>hypothesis #2:</h3>
in order to know the chronology or the spread of thought the data should not only one hour time frame. it's supposed to be more than one hour time frame.

<h3>hypothesis #3:</h3>


In [ ]:
#compiled_data[pd.isnull(compiled_data['retweeted_status'])==False]['retweeted_status']
#print(len(compiled_data))
#compiled_data.loc[4363]

wordfreq=[]

txt=TextBlob(compiled_data.iloc[0]['hashtags']).words
wordfreq = [txt.count(p) for p in txt]
zz = [compiled_data.iloc[0]['p_pos'] for x in txt]

wordfreq_dict=dict(zip(txt,zz))